`https://pytorch.org/ignite/concepts.html`

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim

from ignite.engine import Engine
from ignite.engine import Events
from ignite.metrics import Accuracy

In [63]:
x = torch.rand(100, 3)
y = torch.rand(100, 1)

model = nn.Linear(in_features=3, out_features=1)
optimizer = optim.Adam(model.parameters())
crit = nn.MSELoss()

n_epochs = 10

In [64]:
from torch.utils.data import Dataset, DataLoader


class testDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        super().__init__()

    def __len__(self):
        return self.data.size(0)

    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.labels[idx]
        return x, y


indices = torch.randperm(x.size(0))
train_cnt = int(x.size(0) * 0.8)
valid_cnt = x.size(0) - train_cnt

train_x, valid_x = torch.index_select(x, dim=0, index=indices).split(
    [train_cnt, valid_cnt], dim=0
)

train_y, valid_y = torch.index_select(y, dim=0, index=indices).split(
    [train_cnt, valid_cnt], dim=0
)


train_loader = DataLoader(
    dataset=testDataset(train_x, train_y), batch_size=16, shuffle=True
)

valid_loader = DataLoader(
    dataset=testDataset(train_y, valid_y), batch_size=16, shuffle=True
)

In [65]:
def train_step(engine, mini_batch):
    model.train()
    optimizer.zero_grad()

    x, y = mini_batch
    y_pred = model(x)
    loss = crit(y_pred, y)
    loss.backward()
    optimizer.step()

    # The type of output of the training step (i.e. loss.item() in the above example)
    # is not restricted. Training step function can return everything user wants.
    # Output is set to trainer.state.output and can be used further for any type of processing.
    return loss.item()


train_engine = Engine(train_step)

train_engine.add_event_handler(Events.STARTED, lambda _: print("Start Training."))


## OR
@train_engine.on(Events.STARTED)
def on_training_started(engine):
    print("Another message of start training.")


def on_iteration_completed(engine):
    iteration = engine.state.iteration
    epoch = engine.state.epoch
    loss = engine.state.output
    print(f"Epoch: {epoch}, Iteration: {iteration}, Loss: {loss}")


train_engine.add_event_handler(Events.ITERATION_COMPLETED, on_iteration_completed)

mydata = [1, 2, 3, 4]


def on_training_ended(engine, data):
    print(f"Training is ended. mydata={data}")


train_engine.add_event_handler(Events.COMPLETED, on_training_ended, mydata)

train_engine.run(train_loader, max_epochs=10)

Start Training.
Another message of start training.
Epoch: 1, Iteration: 1, Loss: 0.8205951452255249
Epoch: 1, Iteration: 2, Loss: 0.8183078169822693
Epoch: 1, Iteration: 3, Loss: 0.8641492128372192
Epoch: 1, Iteration: 4, Loss: 0.8837684988975525
Epoch: 1, Iteration: 5, Loss: 0.9812270402908325
Epoch: 2, Iteration: 6, Loss: 0.9132032990455627
Epoch: 2, Iteration: 7, Loss: 0.7664456367492676
Epoch: 2, Iteration: 8, Loss: 0.8986122012138367
Epoch: 2, Iteration: 9, Loss: 0.7346012592315674
Epoch: 2, Iteration: 10, Loss: 0.9507958889007568
Epoch: 3, Iteration: 11, Loss: 0.8530470132827759
Epoch: 3, Iteration: 12, Loss: 0.7313729524612427
Epoch: 3, Iteration: 13, Loss: 0.7436090707778931
Epoch: 3, Iteration: 14, Loss: 0.9435837268829346
Epoch: 3, Iteration: 15, Loss: 0.8863449692726135
Epoch: 4, Iteration: 16, Loss: 0.9313111901283264
Epoch: 4, Iteration: 17, Loss: 0.7555874586105347
Epoch: 4, Iteration: 18, Loss: 0.6950225830078125
Epoch: 4, Iteration: 19, Loss: 0.8302692770957947
Epoch: 4

State:
	iteration: 50
	epoch: 10
	epoch_length: 5
	max_epochs: 10
	output: 0.658409833908081
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>